In [1]:
#IMPORT
import random
import hashlib
from itertools import chain
import hashlib
import binascii
import sys

In [2]:
#PARAMS
p = 31
Fp = GF(p)
n = 256
k = 52
r = n-k
t = n
z = 3
Fz = GF(z)
lamb = 128
lev_leaf = log(t,2)
Fp_set = Set(Fp)
Fp_star = Fp_set.difference([0])

In [3]:
#FUNZIONI DI UTILITA

def generate_random_seed():
    set_random_seed()
    return initial_seed()

def set_seed(seed):
    set_random_seed(seed)
    return initial_seed()

def hash_and_get_seed(input):
    hash = hashlib.sha256(str(input).encode()).digest()
    seed = int.from_bytes(hash, 'big')
    return seed

def hexhash(input):
    return hashlib.sha256(input.encode()).hexdigest()

def convert_int_to_hash_string(input):
    return input.to_bytes((input.bit_length() + 7) // 8, 'big').hex()

def expand_seed(seed_init):
    seed_str = str(seed_init)
    seeds = hashlib.sha256(seed_str.encode()).digest()
    
    seed_1 = seeds[:16] #i primi 16 byte
    seed_2 = seeds[16:] #gli ultimi 16 byte
    
    seed_1 = int.from_bytes(seed_1, 'big')
    
    seed_2 = int.from_bytes(seed_2, 'big')
    
    return seed_1, seed_2
#t=8
# #w = 5

#0,2,4,5,7

#Creo Generatore g del Campo E
def get_generator(p, z, Fp):
    assert is_prime(p),"p is not prime!"
    assert (p-1)%z == 0, "z does not divide p-1!"
    alpha = Fp.primitive_element()
    #non ricordo perchè exp e g sono scelti secondo questa logica
    exp = (p-1)/z
    g = alpha**exp
    return g

def create_random_vector(domain, n_elems, seed):
    set_seed(seed)
    return random_vector(domain, n_elems)

def create_random_matrix(domain, n_rows, n_columns, seed):
    set_seed(seed)
    M = random_matrix(domain, n_rows, n_columns)
    return M

#Creo vettore errore con elementi in E e lunghezza V 
def create_vEn(g, n, Fz, Fp, seed):
    eta = create_random_vector(Fz, n, seed)
    e = vector(Fp, n)
    for i in range(0, n):#range check
        e[i] = (g**eta[i])
    return e

def create_vector_over_Fp_star(Fp_star, t, seed):
    set_random_seed(seed)
    return [Fp_star.random_element() for i in range(t)]

#def create_ch2_vector(t, seed):

def componentwise_multiply(v1, v2):
    if len(v1) != len(v2):
        raise ValueError("I vettori devono avere la stessa dimensione")
    
    result = vector(GF(p), len(v1))  # Crea un nuovo vettore per i risultati
    for i in range(len(v1)):
        result[i] = v1[i] * v2[i]  # Moltiplica gli elementi corrispondenti
    
    return result

In [4]:
#SEED TREE
class SeedTree:
    def __init__(self, depth, salt, seed_m):
        if(depth>0):
            self.seed_m = seed_m
            self.depth = depth
            self.salt = str(salt)
            self.tree = None
            self.full = False
            self.seed_path = None
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self):
        if self.full:
            raise Exception('Albero gia creato')
        else:
            #BISOGNA AGGIUNGERE CONTROLLI?
            self.tree = []
            self.__make_tree(self.seed_m, 0)
            self.full = True
        
    def get_leaves(self):
        if self.full:
            return self.tree[self.depth]
        else:
            raise Exception('Albero non creato')

    def get_seed_path(self):
        if self.full and self.seed_path is not None:
            return self.seed_path

    def compute_seed_path(self, list_nodes):
        if self.full and len(list_nodes)>0:
            self.seed_path={}
            self.__compute_seed_path(self.depth-1, list_nodes)
            return self.seed_path
        else:
            raise Exception('Albero non creato o foglia non valida')

#DA FARE
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth<=self.depth):
            list_children = self.__create_new_level(list_nodes, depth)
            self.tree.append(list_children)
            self.__make_tree(list_children, depth+1)

    def __create_new_level(self, parents, level):
        children = []
        if level == 0:
            sx, dx = self.__compute_children_value(parents, '')
            children.append(sx)
            children.append(dx)
        else:
            base = 2**level
            for i in range(0, len(parents)):
                sx, dx = self.__compute_children_value(parents[i], base+i)
                children.append(sx)
                children.append(dx)
        return children
        
    def __compute_children_value(self, seed_parent, index_parent):
        nodes = str(seed_parent) + self.salt + str(index_parent)
        sx, dx = expand_seed(int(nodes))  
        return sx, dx
    
    #provvisorio
    def __compute_seed_path(self, depth, list_index):
        if depth>0:
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))#prendo gli indici dei nodi padri eliminando i duplicati
            list_sub_root = list(filter(lambda x: ((x % 2 == 0 and x + 1 not in list_parent) or (x % 2 != 0 and x - 1 not in list_parent)), list_parent))
            list_new_children = list(filter(lambda x:  x not in list_sub_root, list_parent))
            if len(list_new_children)>0:
                self.seed_path[depth] = dict(map(lambda x: (x, self.tree[depth][x]), list_new_children))
                self.__compute_seed_path(depth-1, list_new_children)
            
        

#DA FARE
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()
 

In [5]:
#MERKLE TREE

class MerkleTree:
    def __init__(self, n_leaves):
        if(n_leaves%2==0):
            self.n_leaves = n_leaves
            self.depth = log(n_leaves, 2)
            self.tree =  None
            self.merkle_proof =  None
            self.full = False
        else:
            raise Exception('Il numero delle foglie deve essere una porenza di 2')

    #METODI PUBBLICI 
    def make_tree(self, leaves):
        if isinstance(leaves, list) and len(leaves) == self.n_leaves and not(self.full):
            self.tree = []
            self.tree.append(leaves)
            self.__make_tree(leaves, self.depth-1)
            self.full = True
        else:
            raise Exception('Il numero delle foglie deve essere pari a {}'.format(self.n_leaves))
       
    def get_root(self):
        if self.full:
            return self.tree[0][0]
        else:
            raise Exception('Albero non creato')

    def get_merkle_proof(self):
        if self.full and self.merkle_proof is not None:
            return self.merkle_proof

    def compute_proof(self, list_nodes):
        if all(self.full or 1 <= node <= 2**self.depth for node in list_nodes):
            list_leaves = list(map(lambda x: x+1 if x % 2 == 0 else x-1, list_nodes))
            list_leaves = list(filter(lambda x: x not in list_nodes, list_leaves))
            list_indexes = self.__get_indexes(self.depth, list_leaves, [])
            self.__compute_merkle_proof(list_indexes)
            return self.merkle_proof
        else:
            raise Exception('Albero non creato o foglia non valida')

#DA FARE
    def verify_proof(self, node, list_proof):
        if self.full:  
            list_proof.append(node)
            return self.__verify_proof(list_proof)
        else:
            raise Exception('Albero non creato')  
    
    #METODI PRIVATI

    def __make_tree(self, list_nodes, depth):
        if(depth>=0):
            list_parent = self.__create_new_level(list_nodes)
            self.tree.insert(0, list_parent)
            self.__make_tree(list_parent, depth-1)

    def __create_new_level(self, child_nodes):
        parents = []
        for i in range(0, len(child_nodes), 2):
            new_node = self.__compute_father_value(child_nodes[i], child_nodes[i+1])
            parents.append(new_node)
        return parents
        
    def __compute_father_value(self, sx, dx):
        return hexhash(sx+dx)
    
    def __get_indexes(self, depth, list_index, list_proof):
        if(depth > 0):
            list_proof.insert(0, list_index)
            list_parent = sorted(list(set(map(lambda elem: ((elem // 2) + 1) if (elem // 2) % 2 == 0 else (elem // 2) - 1, list_index))))
            return self.__get_indexes(depth-1, list_parent, list_proof)
        else:
            return list_proof


    def __compute_merkle_proof(self, list_merkle_proofs):
        self.merkle_proof = {}    
        for i in range(len(list_merkle_proofs)): 
            current_proof = list_merkle_proofs[i] #controllo livello i esimo
            if i < len(list_merkle_proofs) - 1:
                next_proof = list_merkle_proofs[i+1]
                current_proof = list(filter(lambda x: x*2 not in next_proof and (x*2)+1 not in next_proof, current_proof))
            if not len(current_proof)==0:
                self.merkle_proof[i+1] = dict(map(lambda x: (x, self.tree[i+1][x]), current_proof))

#DA FARE
    def __verify_proof(self, list_proof, index):
        while len(list_proof) > 1:
            ultimo = list_proof.pop()
            penultimo = list_proof.pop()
            to_hash= penultimo +ultimo
            hash=hashlib.sha256(to_hash.encode()).hexdigest()
            list_proof.append(hash)
        return list_proof[0] == self.get_root()

In [6]:
#GENERAZIONE CHIAVI
def key_gen():
    seed_sk = generate_random_seed() 
    seed_e , seed_pk = expand_seed(seed_sk)
    V = create_random_matrix(Fp, k, r, seed_pk) 
    g = get_generator(p, z, Fp)
    e = create_vEn(g, n, Fz, Fp, seed_e)
    s = e[0:r] + e[r:n]*V
    return seed_sk, seed_pk, s


In [7]:

class Signer:
    def __init__(self, seed_sk, seed_pk, s):
        self.seed_sk = seed_sk
        self.seed_pk = seed_pk
        self.seed_e = None
        self.salt = None
        self.V = None
        self.eta = None
        self.s = s
        self.seed_tree = None
        self.merkle_tree = None
        self.c0 = None
        self.c1 = None
        self.ch1 = None
        self.ch2 = None
        self.h = None



    def key_material_expansion(self):
        self.seed_e , self.seed_pk = expand_seed(self.seed_sk)
        self.V = create_random_matrix(Fp, k, r, self.seed_pk)
        self.eta = create_random_vector(Fz, n, self.seed_e)

    def compute_commitment(self):
        self.seed_m = generate_random_seed() 
        seed_salt = generate_random_seed()
        self.salt = hash_and_get_seed(seed_salt)
        self.seed_tree = SeedTree(lev_leaf, self.salt, self.seed_m)
        self.seed_tree.make_tree()
        self.seeds = self.seed_tree.get_leaves()
        cmt0 = [] 
        cmt1 = []
        g = get_generator(p, z, Fp)
        for i in range(0, t):
            seed_ui, seed_ei = expand_seed(self.seeds[i])
            eta_i = create_random_vector(Fz, n, seed_ei)
            sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
            v = vector(Fp,n) #v∈ Fp^n
            for j in range(0, n-1):
                v[j] = (g**sigma_i[j])  
            ui = create_random_vector(Fp, n, seed_ui)
            u = componentwise_multiply(v, ui)
            st = u[0:r] + u[r:n]*self.V
            commitment0 = hexhash( str(st) + str(sigma_i) + str(self.salt) + str(i) )
            commitment1 = hexhash( str(self.seeds[i]) + str(self.salt) + str(i) )
            cmt0.append(commitment0) 
            cmt1.append(commitment1)
        self.merkle_tree = MerkleTree(t)
        self.merkle_tree.make_tree(cmt0)
        self.c0 = self.merkle_tree.get_root()
        input_c1 = "".join(map(str, cmt1))
        self.c1 = hexhash(input_c1)

    def generate_first_challenge(self, msg):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch1 = self.c0+self.c1+msg+hash_salt
        seed_ch1 = hash_and_get_seed(input_ch1)
        self.ch1 = create_vector_over_Fp_star(Fp_star, t, seed_ch1)

    def generate_first_response(self):
        list_hash_yi=[]
        for seed_i, ch_i_1 in zip(self.seeds, self.ch1):
            seed_ui, seed_ei = expand_seed(seed_i)
            g = get_generator(p, z, Fp)
            ei = create_vEn(g, n, Fz, Fp, seed_ei)
            ui = create_random_vector(Fp, n, seed_ui)
            yi = ui+(ch_i_1)*ei
            hi = hexhash(str(yi))
            list_hash_yi.append(hi)
        self.h = hexhash("".join(list_hash_yi))

    def generate_second_challenge(self, msg):#, seed_tree):
        hash_salt = convert_int_to_hash_string(self.salt)
        input_ch2 = self.c0+self.c1+"".join(map(str, self.ch1))+self.h+msg+hash_salt
        seed_ch2 = hash_and_get_seed(input_ch2)
        self.ch2 = create_random_vector(GF(2), t, seed_ch2)
        #self.ch2 = create_ch2_vector(t, seed_ch2)

    def generate_second_response(self):
        index_1, index_0 = [], []
        rsp0 = []
        rsp1 = []
        g = get_generator(p, z, Fp)
        for i, x in enumerate(self.ch2):
            if x == 0:
                index_0.append(i)
                seed_ui, seed_ei = expand_seed(self.seeds[i])
                eta_i = create_random_vector(Fz, n, seed_ei)
                sigma_i = self.eta-eta_i #sigma_i ∈ Fz^ns
                ei = create_vEn(g, n, Fz, Fp, seed_ei)
                ui = create_random_vector(Fp, n, seed_ui)
                yi = ui+(self.ch1[i])*ei
                rsp0.append([yi, sigma_i])
            else:
                index_1.append(i)
                rsp1.append(hexhash( str(self.seeds[i]) + str(self.salt) + str(i) ))
        [index_1.append(i) if x == 1 else index_0.append(i) for i, x in enumerate(self.ch2)] 
        seed_path = self.seed_tree.compute_seed_path(index_1)
        merkle_proofs = self.merkle_tree.compute_proof(index_0)
        return merkle_proofs, seed_path, rsp0, rsp1

#    def pseudo():
        #for i ← 0 to t − 1 do
            #if (b[i] = 0) then
                #cmt0[i] is recomputed by the verifier, cmt1[i] must be sent
                #rsp0[j] ← (yi, δi)  rsp0[j] ← (yi, σi)
                #rsp1[j] ← cmt1[i]
                #j ← j + 1
            #end
        #end
    def sign_msg(self, msg):
        self.key_material_expansion()
        self.compute_commitment()
        self.generate_first_challenge(msg)
        self.generate_first_response()
        self.generate_second_challenge(msg)
        res = self.generate_second_response()
        for r in res:
            print(r)


In [8]:
seed_sk, seed_pk, s = key_gen()
signer = Signer(seed_sk, seed_pk, s)
signer.sign_msg("buonasera")

{4: {1: '4db7d2f077d7e3aa3484aa6bfefddc4dc64d19397032f030dae1c31b8302d55c'}, 5: {6: 'd5fbd81086c4e61d8ff5bdee6fb4838d517f1b8887780a94d8e09e08c613be33', 14: '3a6f684418bfcd46f4c602e7f0df6b1dc8e41f0f12a447c33f14bc62b7ba60b2'}, 6: {0: 'b6bf1ccbae8030672733eebfb450534fb7ef27c96a195c6d7844ef320ab89c4c', 2: '55186ae2090628e831271cde4b4e518aa34e7756c5895124a7f1acff45ad2b56', 15: 'd52436d2e62ad04ecf817ef3028439f5c893891cf2aa1d0922de51f0d2f01b4a', 23: 'f35bac254549a77276d742332b9c6a43a3964a825ed4137a8bd12e7c4a333d33', 24: 'b28c7ed1dbfd8b8a5ca05df095acda8d5e3404a05f4990255f5a6f201bed38fc', 30: '6381fd40e94305dd500aa2032787759233f7f3bddc1b119abfa55730f827f92c', 36: '10b2d06b0d46f2f3e21605c679c634365c6772648caeb168607c60d5fc83793c', 47: '05632511888cbb46969d610220911492d0281c05764c6d1b28b219afeabc7210', 48: '0934e8c4c782bf3fa26a1d407bda58ca1d4c37857b307ae94faeb5aa26d43cb8', 55: 'bd37ad374aa117b73a808af33d7e69d0997c4f54d220b4a16d6c0b9f0291943b', 59: 'ae118526c56c9ff6b3934c4b8098fcac641a5957648346c8

In [9]:
#Commitment 0
print(signer.c0)  


1ccbdc784371e7794b7b85a8c7597f29eb9399d08fa32b62aed13a0e52cdf026


In [10]:
#Commitment 1
print(signer.c1) 



ed9a289bfca69bedcafa54b0be4eea80bf3ab65781481647295f5a74b75801db


In [11]:
#Challenge 1
print(signer.ch1) 


[12, 17, 4, 30, 18, 18, 13, 23, 13, 10, 19, 2, 14, 1, 9, 11, 6, 30, 27, 5, 24, 2, 29, 22, 15, 15, 27, 29, 4, 15, 28, 26, 26, 16, 16, 15, 27, 8, 20, 18, 19, 16, 20, 19, 7, 4, 24, 20, 6, 15, 20, 21, 30, 5, 1, 15, 6, 29, 19, 15, 19, 16, 15, 10, 13, 10, 24, 4, 20, 18, 27, 17, 27, 5, 27, 22, 10, 20, 25, 8, 12, 22, 5, 29, 4, 28, 15, 25, 12, 14, 26, 10, 29, 22, 26, 5, 28, 6, 13, 29, 19, 11, 14, 10, 16, 11, 29, 21, 13, 7, 26, 29, 16, 14, 30, 24, 7, 19, 3, 9, 12, 19, 14, 8, 1, 13, 25, 29, 7, 15, 16, 13, 28, 3, 25, 2, 1, 17, 12, 7, 7, 28, 8, 24, 8, 29, 28, 27, 14, 10, 16, 28, 22, 17, 7, 18, 21, 5, 8, 20, 6, 9, 8, 24, 26, 10, 16, 13, 18, 5, 19, 4, 26, 24, 26, 23, 8, 25, 15, 15, 19, 13, 20, 11, 26, 18, 29, 13, 24, 17, 26, 27, 30, 12, 21, 29, 8, 19, 8, 22, 4, 8, 4, 13, 12, 23, 7, 12, 11, 21, 10, 29, 20, 8, 26, 3, 23, 23, 23, 11, 23, 4, 8, 10, 29, 25, 10, 23, 4, 5, 13, 23, 28, 10, 6, 12, 5, 24, 18, 17, 9, 30, 23, 15, 2, 15, 9, 1, 6, 28, 15, 4, 28, 5, 11, 14]


In [12]:
#Hash 1
print(signer.h)


a31f665b09c08f214c0d523a55bbf05815d7c9fb5332b4e72b4ef17d808f50af


In [13]:
#Challenge 2
print(signer.ch2) 


(1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0)


In [14]:
#Seed Path che contiene l'insieme dei nodi dei sottoalberi per ricostruire i seed (NON SO SE GIUSTO)
#è stata fatta come un dizionario chiave valore
seed_path = signer.seed_tree.get_seed_path()
size_in_bytes = sys.getsizeof(seed_path)
size_in_bits = size_in_bytes * 8
print(size_in_bits)
for key, value in seed_path.items():
    print("Livello",key)
    print("Lista seed", value)



1984
Livello 7
Lista seed {0: 93609296216569168299382570640971525472, 1: 77887344759631324872256126459004255308, 2: 43009614552921413270372482001062712039, 3: 222553166404443653923228621166389069757, 6: 8851366956070174639836212202282939553, 7: 199200405679078380733920356852699003006, 8: 71783037737399157847542248000445073645, 9: 58533346091192238193289571428977647864, 10: 330218684163035962025918905576303571066, 11: 174254110875033320328760637403159452349, 16: 145888525384411631979399142559786736286, 17: 71755702939952317444616249061357979534, 22: 258195350435017018323492880520178335561, 23: 255549678479940269562100040797466760362, 24: 314739257058840138449645032603564007338, 25: 258366929533412387487321618804468531367, 28: 216679309564009006484068222695824283853, 29: 262103333176870107612630563703344958695, 32: 311923858682963152267066929497216765200, 33: 269620374895973155870902642342020551705, 34: 170093966038401665775168102175937847231, 35: 161475176484413812133342715424326531378,

In [15]:
#Merkle proof che contiene l'insieme dei nodi dei sottoalberi per ricostruire le foglie (DA GLI STESSI RISULTATI DEL TUO) 
#è stato fatta come un dizionario chiave valore
merkle_proof = (signer.merkle_tree.get_merkle_proof())
size_in_bytes = sys.getsizeof(merkle_proof)
size_in_bits = size_in_bytes * 8
print(size_in_bits)
for key, value in merkle_proof.items():
    print("Livello",key)
    print("Lista hash", value)

1984
Livello 4
Lista hash {1: '4db7d2f077d7e3aa3484aa6bfefddc4dc64d19397032f030dae1c31b8302d55c'}
Livello 5
Lista hash {6: 'd5fbd81086c4e61d8ff5bdee6fb4838d517f1b8887780a94d8e09e08c613be33', 14: '3a6f684418bfcd46f4c602e7f0df6b1dc8e41f0f12a447c33f14bc62b7ba60b2'}
Livello 6
Lista hash {0: 'b6bf1ccbae8030672733eebfb450534fb7ef27c96a195c6d7844ef320ab89c4c', 2: '55186ae2090628e831271cde4b4e518aa34e7756c5895124a7f1acff45ad2b56', 15: 'd52436d2e62ad04ecf817ef3028439f5c893891cf2aa1d0922de51f0d2f01b4a', 23: 'f35bac254549a77276d742332b9c6a43a3964a825ed4137a8bd12e7c4a333d33', 24: 'b28c7ed1dbfd8b8a5ca05df095acda8d5e3404a05f4990255f5a6f201bed38fc', 30: '6381fd40e94305dd500aa2032787759233f7f3bddc1b119abfa55730f827f92c', 36: '10b2d06b0d46f2f3e21605c679c634365c6772648caeb168607c60d5fc83793c', 47: '05632511888cbb46969d610220911492d0281c05764c6d1b28b219afeabc7210', 48: '0934e8c4c782bf3fa26a1d407bda58ca1d4c37857b307ae94faeb5aa26d43cb8', 55: 'bd37ad374aa117b73a808af33d7e69d0997c4f54d220b4a16d6c0b9f0291943b